In [1]:
import requests
from bs4 import BeautifulSoup
import lxml
import time
import pandas as pd
from openpyxl import workbook

In [2]:
# org = ["aws","facebook","google","naver","kakao","apple","alibaba","tencent","baidu","microsoft","samsung"]
org = list(input("검색하실 기업이름을 영어로 입력해주세요.(여러개 인경우 띄어쓰기로 구분하여 입력) : ").split())
org_dic={}
for o in org:
    url = "https://github.com/orgs/{}/repositories".format(o)
    print("{} 에 대한 정보 수집 시작.".format(url))
    res= requests.get(url)
    try:
        res.raise_for_status()
    except:
        print("입력하신 기업\"{}\" 에 대한 정보가 존재하지 않습니다.\n".format(o))
        continue
    soup=BeautifulSoup(res.text,"lxml")
    try:
        max_page = int(soup.find("div",attrs={"role":"navigation"}).find_all("a")[-2].get_text())
    except:
        max_page = 1
#     print(o,max_page)
    item_temp = []
    for p in range(1,max_page+1):
        time.sleep(1)
        url = "https://github.com/orgs/{}/repositories?page={}".format(o,p)
        res= requests.get(url)
        res.raise_for_status()
        soup=BeautifulSoup(res.text,"lxml")
        print("{}{}수집시작{}".format("*"*10,o,"*"*10))
        for item in soup.find("div",attrs={"class":"org-repos repo-list"}).find_all("li",attrs={"class":"Box-row"}):
            print(item.a.get_text().strip())
            item_temp.append(item.a.get_text().strip())
    org_dic[o]=item_temp
    print()
        

검색하실 기업이름을 영어로 입력해주세요.(여러개 인경우 띄어쓰기로 구분하여 입력) : intel
https://github.com/orgs/intel/repositories 에 대한 정보 수집 시작.
**********intel수집시작**********
ivsc-driver
auto-pts
llvm
DevCloudContent-helm
intel-device-plugins-for-kubernetes
media-driver
edge-conductor
e2eAIOK
tdx-tools
intel-inb-manageability
wult
llvm-test-suite
intel-sgx-ssl
libva
cri-resource-manager
xpumanager
vaapi-fits
tdx
ehsm
pailliercryptolib_python
lkp-tests
GPGMM
confidential-computing-zoo
linux-kernel-overlay
linux-intel-quilt
linux-intel-lts
llvm-docs
compile-time-init-build
mlir-extensions
thunderbolt-software-kernel-tree
**********intel수집시작**********
intel-graphics-compiler
cve-bin-tool
ideep
linux-sgx
compute-runtime
onnxruntime
pmem-csi
openfl
ipmctl
opencl-intercept-layer
pm-graph
hexl
media-delivery
he-toolkit
CacheLib
idxd-config
fpga-runtime-for-opencl
compute-aggregation-layer
device-modeling-language
QAT_Engine
platform-aware-scheduling
ros2_openvino_toolkit
multimodal-data-visualization
cm-compiler
SGXDataCente

**********intel수집시작**********
envoy
istio-proxy
istio-api
envoy-go-control-plane
psst
libaji_client
aji_openocd
endurance_profiler
xess
goresctrl
oneapi-containers
irqbalance
edge-ai-device-plugin
PCM
flexmalloc
numatop
kvm-unit-tests-tdx
shim-tdx
pinball2elf
dimgen
DML
compute-samples
iefs-kernel-updates
meta-intel-sgx
dml-lang
vbh
user-container-selinux
qemu-dcp
vmtaint
fpga-partial-reconfig
**********intel수집시작**********
intel-sdsi
dcp-overlay
ad-rss-lib
.github
gdb
seam-loader
config-builder
RetailWorkloadOrchestrator
idxd-driver
tdx-module
intel-sdptool-ansible-modules
dsa-perf-micros
vscode-oneapi-devcloud-connector
vscode-oneapi-analysis-configurator
intel-vaapi-driver
RAAD
fpga-npu
Board-Instrumentation-Framework
afxdp-plugins-for-kubernetes
pse-fw
machinepack-mssql
observability-telegraf
libmei
metee
intel-ethernet-operator
code-base-investigator
scale-testing-for-vastdata
S0ixSelftestTool
ioprof
edge-ai-collectd
**********intel수집시작**********
forLAGraphGo
forGraphBLASGo
forGoPa

**********intel수집시작**********
libwsbm
psb_headers
appframeworkPlugins
af-range
toucheffect
cloud-neard



In [3]:
for i in org_dic:
    temp=[]
    for a in org_dic[i]:
        url = "https://github.com/{}/{}".format(i,a)
        
        try:
            res= requests.get(url)
            res.raise_for_status()
            soup=BeautifulSoup(res.text,"lxml")
        except:
            time.sleep(10)
            res= requests.get(url)
            soup=BeautifulSoup(res.text,"lxml")
            
        # 브랜치 수(파생 프로젝트 수)
        try:
            branch_num = soup.find("div",attrs={"class":"flex-self-center ml-3 flex-self-stretch d-none d-lg-flex flex-items-center lh-condensed-ultra"}).strong.get_text()
        except:
            branch_num = 0

        # 풀 리퀘스트 수(코드 병합 요구 수)
        try:
            pull_requests_num = soup.find("a",attrs={"id":"pull-requests-tab"}).find("span",attrs={"class":"Counter"}).get_text()
        except:
            pull_requests_num = 0
            
        # 커밋 수(수정 요구수)
        try:
            commit_num = soup.find("li",attrs={"class":"ml-0 ml-md-3"}).strong.get_text()
        except:
            commit_num = 0
            
        # 스타 수(북마크 수, 인기도)
        try:
            star_num = soup.find("ul",attrs={"class":"pagehead-actions flex-shrink-0 d-none d-md-inline"}).find("a",attrs={"class":"social-count js-social-count"}).get_text()
            star_num = star_num.replace('\t', '').replace('\n', '').strip()
        except:
            star_num = 0
            
        # 토픽(개발언어의 주제)
        try:
            topic = soup.find("div",attrs={"class":"BorderGrid-cell"}).find_all("a",attrs={"class":"topic-tag topic-tag-link"})
            topics=[i.get_text().replace("\n","").replace("\t","").strip() for i in topic]
        except:
            topics = []
            
        # 릴리즈 정보(버전 정보)
        try:
            release_info = soup.find("div",attrs={"class":"BorderGrid BorderGrid--spacious"}).find("span",attrs={"class":"css-truncate css-truncate-target text-bold mr-2"}).get_text().strip()
        except:
            release_info = ""

        temp.append([a,branch_num,pull_requests_num,commit_num,star_num,topics,release_info])
    df = pd.DataFrame(temp,columns=["ProjectName","BranchNum","PullRequestNum","CommitNum","StarNum","Topics","ReleaseInfo"])
    df.to_excel("{}_renew.xlsx".format(i),index=False)
    print("{}_renew.xlsx".format(i),"저장 완료")

KeyboardInterrupt: 